# Extract accessions from Excel table:
Read the table:

In [1]:
import pandas as pd

table = pd.read_excel('Supplementary_Table_1.xlsx',
                      engine='openpyxl')

Get BioProject of interest:

In [2]:
prj = 'PRJNA612766'

acc_df = (
    table.query('BioProject == @prj')
    [['Run', 'Sample Name', 'Collection_Date', 'Host', 'collected_by']]
    )

acc_df.groupby(['Host', 'collected_by', 'Collection_Date']).size()

Host          collected_by                         Collection_Date
Homo sapiens  Renmin Hospital of Wuhan University  12-Feb-2020         31
                                                   15-Jan-2020          2
                                                   30-Jan-2020         96
Plasmid       Aisi Fu                              12-Feb-2020        112
dtype: int64

Write configuration YAMLs for each host:

In [3]:
for host, df in acc_df.groupby('Host'):
    lines = ['# -----------------------------------------------------------------',
             f"# BioProject {prj}, host {host}",
             '# -----------------------------------------------------------------',
             ]
    for _, row in df.iterrows():
        d = row.to_dict()
        lines += [f"{d['Sample Name']}_{prj}:",
                  f"  accessions: [{d['Run']}]",
                  f"  collected_by: {d['collected_by']}",
                  f"  collection_date: {d['Collection_Date']}"
                  ]
    with open(f"{host}_config.yml", 'w') as f:
        f.write('  ' + '\n  '.join(lines))